# Environment Setup

In [1]:
### Imports ###
# General
import urllib.request, json # Downloading Data
import urllib3.contrib.pyopenssl
import numpy as np
import time
import datetime
import math
import random

# File
import pickle

#Web Scraping
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import re, string


### Declarations ###
Api_Key = "API_KEYs"
with open('RiotAPIKey.txt') as fp:
    Api_Key = fp.read()
gameData = np.array([])
userData = {}

_SEED_USER = "Faendol" # User To Start the Search At
_USER_DEPTH = 1 # Numbers of 'Jumps' out to spread search to. Gets a lot more data but takes a very long time

### Web Functions ###
TimeStart = 0
totalrequests = 0

def getLeagueData(urlData, depth=0): # Function for querying Riot API with Rate Limiting
    global TimeStart, totalrequests
    # Check if Need to reset requests counter
    if (TimeStart + 120) <= int(time.time()):
        TimeStart = int(time.time())
    
    #Check if over requests
    if totalrequests > 99:
        print("  Over Requests! Sleeping " + str((TimeStart + 120) - int(time.time())) + " Seconds", end='', flush=True)
        if math.ceil((TimeStart + 120) - int(time.time())) > 0:
            time.sleep(math.ceil((TimeStart + 122) - int(time.time()))) # Add 2 seconds because Riot API Seems to be slightly off
        totalrequests = 0
        TimeStart = int(time.time())
    # Get Data
    try:
        with urllib.request.urlopen(urlData) as url:
            for header in url.getheaders():
                if header[0] == "X-App-Rate-Limit-Count":
                    tempstr = header[1]
            totalrequests = int(tempstr[tempstr.index(",")+1:tempstr.rfind(":")])
            return json.loads(url.read().decode())
    except urllib.error.HTTPError as error:
        if error.code == 429:
            print("Queries: {}\nTime1: {}\nTime2: {}".format(totalrequests, (TimeStart + 120) - int(time.time()), error.getheaders()[2][1]))
            time.sleep(int(error.getheaders()[2][1]))
            totalrequests = 0
            TimeStart = int(time.time())
            return getLeagueData(urlData)
        else:
            if error.code == 500 or error.code == 503:
                print('Server Error ' + urlData)
                time.sleep(10)
                return getLeagueData(urlData)
            else:
                if error.code == 404 and depth < 4: # Check if actual 404 or Riot API error
                    print("Potential Server Error" + urlData)
                    time.sleep(30)
                    return getLeagueData(urlData, depth + 1)
                else:
                    print(urlData)
                    raise
                
### Get and Update Game Data ###
def updateGameData(localData, extra = ""):
    global playerData, gameData
    
    LeagueDataCount = 1
    ## Get All Match Data for Each Match
    for dataPoint in localData["matches"]:
        print('\r' + str(LeagueDataCount) + ' / ' + str(len(localData['matches'])) + " " + str(extra), end='', flush=True)
        LeagueDataCount += 1
        
        if any(x['gameID'] == dataPoint['gameId'] for x in gameData): #Check match not already evaluated
            continue
        
        data = getLeagueData("https://na1.api.riotgames.com/lol/match/v4/matches/" + str(dataPoint['gameId']) + "?api_key=" + Api_Key)
        
        dataobj = {
            'gameID': dataPoint['gameId'],
            'winner': data['teams'][0]['win'] == 'Win', #true for team 100
            'players': []
        }
        
        for player in data['participantIdentities']:
            playerData = {
                'playerID': player['participantId'],
                'summonerID': player['player']['summonerId'],
                'accountID': player['player']['accountId'],
                'currentAccountID': player['player']['currentAccountId'],
                'championID': data['participants'][int(player['participantId'])-1]['championId'],
                'lane': data['participants'][int(player['participantId'])-1]['timeline']['lane']
            }
            
            if playerData['accountID'] not in userData:
                userData[playerData['accountID']] = {
                    'playerID': player['participantId'],
                    'summonerID': player['player']['summonerId'],
                    'accountID': player['player']['accountId'],
                    'currentAccountID': player['player']['currentAccountId'],
                    'level': -1,
                    'rank': -1,
                    'champions': {},
                    'gotMatchHistory': False
                    }
            
            dataobj['players'].append(playerData)
        
        gameData = np.append(gameData, dataobj)

# NA.OP.GG champion winrates

In [2]:
### Scrape all winrates from na.op.gg
# Get Champion Names and IDs
ChampMap = {}
pattern = re.compile('[\W_]+')

version = "1.1"
with closing(get("https://ddragon.leagueoflegends.com/api/versions.json", stream=True)) as resp:
    with closing(get("http://ddragon.leagueoflegends.com/cdn/{0}/data/en_US/champion.json".format(json.loads(resp.content)[0]), stream=True)) as rep:
        localData = json.loads(rep.content)["data"]
        for champ in localData:
            ChampMap[pattern.sub('', localData[champ]["name"].lower())] = localData[champ]['key']
    
webResponse = None
ChampData = {}

try:
    with closing(get("https://na.op.gg/statistics/ajax2/champion/type=win&league=&period=month&mapId=12&queue=aram", stream=True)) as resp:
        webResponse = BeautifulSoup(resp.content, 'html.parser')

except RequestException as e:
    log_error('Error during requests to na.op.gg : {0}'.format(str(e)))

parent = webResponse.select("tbody")[0].find_all("tr")

for tr in parent:
    ChampData[int(ChampMap[pattern.sub('', tr.find("td", class_="Cell ChampionName").a.string.lower())])] = (float(tr.contents[7]["data-value"]) / 100)

print('Calculated Winrates')

Calculated Winrates


# Get all games from seed user

In [3]:
# Get User Information
seedUserData = getLeagueData("https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + _SEED_USER + "?api_key=" + Api_Key)

## Get All Matches For Specific Account ID
updateGameData(getLeagueData("https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + seedUserData["accountId"] + "?queue=450&api_key=" + Api_Key))
print("\nGot Initial Game Data")

100 / 100  Over Requests! Sleeping 64 Seconds
Got Initial Game Data


# Expand Search and Get All Games for All Users

In [4]:
TotalUserCount = 1

for i in range(_USER_DEPTH):
    for user in list(userData):
        if userData[user]['gotMatchHistory'] == False:
            updateGameData(getLeagueData("https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + str(userData[user]['accountID']) + "?queue=450&api_key=" + Api_Key), " -- {} / {}".format(TotalUserCount, len(userData)))
            userData[user]['gotMatchHistory'] = True
        TotalUserCount += 1

36 / 60  -- 12 / 882359ver Requests! Sleeping 63 Seconds  Over Requests! Sleeping 65 Seconds  Over Requests! Sleeping 65 Seconds  Over Requests! Sleeping 64 Seconds  Over Requests! Sleeping 65 Seconds  Over Requests! Sleeping 65 Seconds  Over Requests! Sleeping 64 Seconds  Over Requests! Sleeping 65 Seconds  Over Requests! Sleeping 65 Seconds  Over Requests! Sleeping 62 SecondsPotential Server Errorhttps://na1.api.riotgames.com/lol/match/v4/matches/3057183148?api_key=RGAPI-1142dbcf-21d8-4c68-94b0-f75f472b06f9
58 / 100  -- 13 / 9129Potential Server Errorhttps://na1.api.riotgames.com/lol/match/v4/matches/3069084037?api_key=RGAPI-1142dbcf-21d8-4c68-94b0-f75f472b06f9
2 / 100  -- 21 / 1443821Over Requests! Sleeping 4 Seconds  Over Requests! Sleeping 63 Seconds  Over Requests! Sleeping 61 Seconds  Over Requests! Sleeping 63 Seconds  Over Requests! Sleeping 63 Seconds  Over Requests! Sleeping 64 Seconds  Over Requests! Sleeping 64 Seconds

KeyboardInterrupt: 

# Get Ranks

In [4]:
# Dictionary for Getting Values
tierPoints = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'PLATINUM','DIAMOND','MASTER','CHALLENGER', 'GRANDMASTER']
rankPoints= ["IV", "III", "II", "I"]

rankCount = 1

# Get User Rank
for user in userData:
    if userData[user]['rank'] >= 0:
        rankCount += 1
        continue
    
    print('\r{} / {}'.format(rankCount, len(userData)), end='')
    tempUserRankData = getLeagueData("https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/" + str(userData[user]['summonerID']) + "?api_key=" + Api_Key)
    if len(tempUserRankData) > 0:
        for rankData in tempUserRankData:
            rank = rankData['rank']
            tier = rankData['tier']
            leaguepoints = rankData['leaguePoints']
            
            # Calculate Point Value from Rank
            tempRank = (tierPoints.index(tier) * 500) + (rankPoints.index(rank) * 100) + int(leaguepoints)
            
            if tempRank > userData[user]['rank']:
                userData[user]['rank'] = tempRank
                userData[user]['rankString'] = '{} {} {} LP'.format(tier, rank, leaguepoints)
    else:
        userData[user]['rank'] = 0
    rankCount += 1
print("\nGot All Ranked Data")

69 / 761Server Error https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/g_GapeqsaXixLzBHEjkV2LMG0C0PnNSsmGkodoWgNxl-iKU?api_key=RGAPI-1142dbcf-21d8-4c68-94b0-f75f472b06f9
761 / 761  Over Requests! Sleeping 76 Seconds
Got All Ranked Data


# Get Level

In [5]:
levelDataCount = 1

# Get User Level
for user in userData:
    if userData[user]['level'] >= 0:
        levelDataCount += 1
        continue
    
    print('\r' + str(levelDataCount) + ' / ' + str(len(userData)), end='', flush=True)
    tempUserLevelData = getLeagueData("https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-account/" + str(userData[user]['currentAccountID']) + "?api_key=" + Api_Key)
    userData[user]['level'] = tempUserLevelData['summonerLevel']
    levelDataCount += 1
print('\nGot all Level Data')

761 / 761 Over Requests! Sleeping 77 Seconds  Over Requests! Sleeping 77 Seconds  Over Requests! Sleeping 77 Seconds  Over Requests! Sleeping 76 Seconds  Over Requests! Sleeping 77 Seconds  Over Requests! Sleeping 77 Seconds  Over Requests! Sleeping 76 Seconds  Over Requests! Sleeping 77 Seconds
Got all Level Data


# Get User Mastery

In [6]:
masteryDataCount = 1

# Get User Mastery
for user in userData:
    if len(userData[user]['champions']) > 0:
        masteryDataCount += 1
        continue
    
    print('\r{} / {} ETA {} hours'.format(masteryDataCount, len(userData), round((len(userData) - masteryDataCount) / 50 / 60, 3)), end='')
    tempUserMasteryData = getLeagueData('https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/' + str(userData[user]['summonerID']) + '?api_key=' + Api_Key)
    for champMastery in tempUserMasteryData:
        userData[user]['champions'][champMastery['championId']] = {
            'championID': champMastery['championId'],
            'level': champMastery['championLevel'],
            'points': champMastery['championPoints']
        }
    masteryDataCount += 1
print("\nGot All Mastery")

761 / 761 ETA 0.0 hoursrs  Over Requests! Sleeping 66 Seconds
Got All Mastery


# Save Data

In [7]:
with open('./Data/GameData' + datetime.datetime.now().strftime(".%Y.%m.%d") + '.pickle', 'wb') as handle:
    pickle.dump(gameData, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./Data/PlayerData' + datetime.datetime.now().strftime(".%Y.%m.%d") + '.pickle', 'wb') as handle:
    pickle.dump(userData, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./DataChampData' + datetime.datetime.now().strftime(".%Y.%m.%d") + '.pickle', 'wb') as handle:
    pickle.dump(ChampData, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print("Saved Data: " + datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y"))

Saved Data: 06:05PM on July 10, 2019
